<a href="https://colab.research.google.com/github/danielglz096/data-analytics/blob/main/modelo_analitica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Analitica de Datos


*   Jose Daniel Gonzalez - A01751681
*   Victor Eduardo Perez - A01275284

Version 1

##Librerias

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
from category_encoders import TargetEncoder
import numpy as np
from scipy import stats

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Preparar Datos para el Modelo

### Demografico

In [19]:
df = pd.read_csv('/content/drive/MyDrive/analitica/reto_final/demografico.csv')
display(df.head(4))

,id,nacimiento,genero,hijos,ingreso,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,salida,antiguedad,edad_salida,Desc Medida,grupo_persona,evaluacion,funcion,distancia_km,tiempo_h
0,70634620,2/28/1984,Hombre,0,8/24/2022,38,2770,234,Suburbia Los Cabos Patio,10/08/22,0,38,Baja Suburbia,SBB Planta,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222
1,70636679,12/23/1990,Mujer,0,8/31/2022,31,11500,234,Suburbia Los Cabos Patio,11/03/22,0,31,Baja Suburbia,SBB Planta,3.0,Supervisor Cajas,18.52999,27.51166667
2,70642442,6/20/1981,Mujer,0,9/28/2022,41,23410,234,Suburbia Los Cabos Patio,11/22/2022,0,41,Baja Suburbia,SBB Planta,3.0,Auxiliar Cajero,13.77464,22.87666667
3,70639827,05/02/93,Hombre,0,9/14/2022,29,23428,234,Suburbia Los Cabos Patio,10/26/2022,0,29,Baja Suburbia,SBB Planta,3.0,Subjefe Proteccion,0.04798,0.221944444


#### Limpieza

In [20]:
def count_missing_values(df):
    # Check for missing values in each column
    missing_values = df.isna().sum()

    # Create a DataFrame to store the results
    missing_info = pd.DataFrame({
        'Column Name': missing_values.index,
        'Null/NaN Count': missing_values.values
    })

    return missing_info


# Call the function to get the missing values information
missing_info = count_missing_values(df)

# Print the results
print(missing_info)


      Column Name  Null/NaN Count
0              id               0
1      nacimiento               0
2          genero               0
3           hijos               0
4         ingreso               0
5    edad_ingreso               0
6     cp_vivienda               0
7      cp_trabajo               0
8       ubicacion               0
9          salida               0
10     antiguedad               0
11    edad_salida               0
12    Desc Medida               0
13  grupo_persona               0
14     evaluacion               0
15        funcion               0
16   distancia_km               0
17       tiempo_h               0


##### Tratar valores atipicos

In [21]:
def treat_atypical_values(df, columns_to_treat, z_threshold=3):
    treated_df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for column in columns_to_treat:
        if column in treated_df.columns:
            # Calculate the z-scores for the selected column
            z_scores = np.abs(stats.zscore(treated_df[column]))

            # Identify outliers
            outliers = treated_df[z_scores > z_threshold]

            # Decide how to treat outliers (e.g., replace with the median)
            median_value = treated_df[column].median()
            treated_df.loc[z_scores > z_threshold, column] = median_value

    return treated_df

In [22]:
columns_to_treat = ['id', 'antiguedad', 'hijos']

clean_df = treat_atypical_values(df, columns_to_treat)

#### Diseño y construccion

##### Fechas

In [23]:
def split_date_column(df, date_column_name, day_column_name, month_column_name, year_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name])  # Convert the date column to DateTime type
    df[day_column_name] = df[date_column_name].dt.day
    df[month_column_name] = df[date_column_name].dt.month
    df[year_column_name] = df[date_column_name].dt.year
    df.drop(columns=[date_column_name], inplace=True)  # Drop the original date column

In [24]:
split_date_column(df,"ingreso","dia_ingreso","mes_ingreso","year_ingreso")
display(df.head(4))

,id,nacimiento,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,salida,antiguedad,edad_salida,Desc Medida,grupo_persona,evaluacion,funcion,distancia_km,tiempo_h,dia_ingreso,mes_ingreso,year_ingreso
0,70634620,2/28/1984,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,10/08/22,0,38,Baja Suburbia,SBB Planta,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222,24,8,2022
1,70636679,12/23/1990,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,11/03/22,0,31,Baja Suburbia,SBB Planta,3.0,Supervisor Cajas,18.52999,27.51166667,31,8,2022
2,70642442,6/20/1981,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,11/22/2022,0,41,Baja Suburbia,SBB Planta,3.0,Auxiliar Cajero,13.77464,22.87666667,28,9,2022
3,70639827,05/02/93,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,10/26/2022,0,29,Baja Suburbia,SBB Planta,3.0,Subjefe Proteccion,0.04798,0.221944444,14,9,2022


In [25]:
split_date_column(df,"salida","dia_salida","mes_salida","year_salida")
display(df.head(4))

,id,nacimiento,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,...,evaluacion,funcion,distancia_km,tiempo_h,dia_ingreso,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida
0,70634620,2/28/1984,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,0,38,...,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222,24,8,2022,8,10,2022
1,70636679,12/23/1990,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,0,31,...,3.0,Supervisor Cajas,18.52999,27.51166667,31,8,2022,3,11,2022
2,70642442,6/20/1981,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,0,41,...,3.0,Auxiliar Cajero,13.77464,22.87666667,28,9,2022,22,11,2022
3,70639827,05/02/93,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,0,29,...,3.0,Subjefe Proteccion,0.04798,0.221944444,14,9,2022,26,10,2022


In [26]:
split_date_column(df,"nacimiento","dia_nacimiento","mes_nacimiento","year_nacimiento")
display(df.head(4))

,id,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,Desc Medida,...,tiempo_h,dia_ingreso,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento
0,70634620,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,0,38,Baja Suburbia,...,25.22972222,24,8,2022,8,10,2022,28,2,1984
1,70636679,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,0,31,Baja Suburbia,...,27.51166667,31,8,2022,3,11,2022,23,12,1990
2,70642442,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,0,41,Baja Suburbia,...,22.87666667,28,9,2022,22,11,2022,20,6,1981
3,70639827,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,0,29,Baja Suburbia,...,0.221944444,14,9,2022,26,10,2022,2,5,1993


##### One hot encoding

###### Genero

In [27]:
df = pd.get_dummies(df, columns=['genero'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,Desc Medida,grupo_persona,...,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento,Hombre,Mujer
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,Baja Suburbia,SBB Planta,...,8,2022,8,10,2022,28,2,1984,1,0
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,Baja Suburbia,SBB Planta,...,8,2022,3,11,2022,23,12,1990,0,1
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,Baja Suburbia,SBB Planta,...,9,2022,22,11,2022,20,6,1981,0,1
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,Baja Suburbia,SBB Planta,...,9,2022,26,10,2022,2,5,1993,1,0


###### Tipo de baja

In [28]:
# Map the values to the desired categories
df['Desc Medida'] = df['Desc Medida'].map({
    'Baja': 'baja_liverpool',
    'Baja por Renuncia Voluntaria': 'baja_liverpool',
    'Baja Suburbia': 'baja_suburbia'
})

# Perform one-hot encoding
df = pd.get_dummies(df, columns=['Desc Medida'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,grupo_persona,evaluacion,...,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento,Hombre,Mujer,baja_liverpool,baja_suburbia
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,SBB Planta,3.0,...,8,10,2022,28,2,1984,1,0,0,1
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,SBB Planta,3.0,...,3,11,2022,23,12,1990,0,1,0,1
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,SBB Planta,3.0,...,22,11,2022,20,6,1981,0,1,0,1
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,SBB Planta,3.0,...,26,10,2022,2,5,1993,1,0,0,1


###### Grupo Persona

In [29]:
df = pd.get_dummies(df, columns=['grupo_persona'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,evaluacion,funcion,...,baja_liverpool,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,3.0,Jefe Prevencion Perdidas,...,0,1,0,0,0,0,0,1,0,0
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,3.0,Supervisor Cajas,...,0,1,0,0,0,0,0,1,0,0
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,3.0,Auxiliar Cajero,...,0,1,0,0,0,0,0,1,0,0
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,3.0,Subjefe Proteccion,...,0,1,0,0,0,0,0,1,0,0


##### Rellenar codigos postales

In [30]:
df['cp_trabajo'] = df['cp_trabajo'].astype(str)
df['cp_trabajo'] = df['cp_trabajo'].str.pad(width=5, side='right', fillchar='0')

df['cp_vivienda'] = df['cp_vivienda'].astype(str)
df['cp_vivienda'] = df['cp_vivienda'].str.pad(width=5, side='right', fillchar='0')

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,evaluacion,funcion,...,baja_liverpool,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.
0,70634620,0,38,27700,23400,Suburbia Los Cabos Patio,0,38,3.0,Jefe Prevencion Perdidas,...,0,1,0,0,0,0,0,1,0,0
1,70636679,0,31,11500,23400,Suburbia Los Cabos Patio,0,31,3.0,Supervisor Cajas,...,0,1,0,0,0,0,0,1,0,0
2,70642442,0,41,23410,23400,Suburbia Los Cabos Patio,0,41,3.0,Auxiliar Cajero,...,0,1,0,0,0,0,0,1,0,0
3,70639827,0,29,23428,23400,Suburbia Los Cabos Patio,0,29,3.0,Subjefe Proteccion,...,0,1,0,0,0,0,0,1,0,0


##### Target Encoding

###### Ubicacion

In [31]:
encoder = TargetEncoder()
encoder.fit(df['ubicacion'], df['antiguedad'])

df['ubicacion_encoded'] = encoder.transform(df['ubicacion'])

df.drop(columns=['ubicacion'],inplace=True)

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,antiguedad,edad_salida,evaluacion,funcion,distancia_km,...,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.,ubicacion_encoded
0,70634620,0,38,27700,23400,0,38,3.0,Jefe Prevencion Perdidas,16.02733,...,1,0,0,0,0,0,1,0,0,1.164766
1,70636679,0,31,11500,23400,0,31,3.0,Supervisor Cajas,18.52999,...,1,0,0,0,0,0,1,0,0,1.164766
2,70642442,0,41,23410,23400,0,41,3.0,Auxiliar Cajero,13.77464,...,1,0,0,0,0,0,1,0,0,1.164766
3,70639827,0,29,23428,23400,0,29,3.0,Subjefe Proteccion,0.04798,...,1,0,0,0,0,0,1,0,0,1.164766


###### Funcion

In [32]:
encoder = TargetEncoder()
encoder.fit(df['funcion'], df['antiguedad'])

df['funcion_encoded'] = encoder.transform(df['funcion'])

df.drop(columns=['funcion'],inplace=True)

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,antiguedad,edad_salida,evaluacion,distancia_km,tiempo_h,...,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.,ubicacion_encoded,funcion_encoded
0,70634620,0,38,27700,23400,0,38,3.0,16.02733,25.22972222,...,0,0,0,0,0,1,0,0,1.164766,2.429671
1,70636679,0,31,11500,23400,0,31,3.0,18.52999,27.51166667,...,0,0,0,0,0,1,0,0,1.164766,2.213115
2,70642442,0,41,23410,23400,0,41,3.0,13.77464,22.87666667,...,0,0,0,0,0,1,0,0,1.164766,0.879807
3,70639827,0,29,23428,23400,0,29,3.0,0.04798,0.221944444,...,0,0,0,0,0,1,0,0,1.164766,2.153846


In [33]:
display(df.columns)

Index(['id', 'hijos', 'edad_ingreso', 'cp_vivienda', 'cp_trabajo',
       'antiguedad', 'edad_salida', 'evaluacion', 'distancia_km', 'tiempo_h',
       'dia_ingreso', 'mes_ingreso', 'year_ingreso', 'dia_salida',
       'mes_salida', 'year_salida', 'dia_nacimiento', 'mes_nacimiento',
       'year_nacimiento', 'Hombre', 'Mujer', 'baja_liverpool', 'baja_suburbia',
       'Eventual No Sind.', 'Eventual Sind.', 'Planta No Sind.',
       'Planta Sind.', 'SBB Eventuales', 'SBB Planta', 'Vía Planta No Sind.',
       'Vía Planta Sind.', 'ubicacion_encoded', 'funcion_encoded'],
      dtype='object')

In [34]:
file_path = '/content/drive/My Drive/analitica/reto_final/demgrafico_transformado.csv'

# Save the DataFrame to the specified file path
df.to_csv(file_path, index=False)

### Historico

In [9]:
df2 = pd.read_csv('/content/drive/MyDrive/analitica/reto_final/historico.csv')
display(df2.head(4))

,id,hasta,desde,sociedad,div_personal,grupo_personal,area_personal,ubicacion,unidad_organizacional,funcion
0,3369,1998-12-31,1974-12-01,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Sin Un Org,Sin Función
1,3369,1999-12-31,1999-01-01,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B
2,3369,2004-01-31,2000-01-01,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B
3,3369,2005-11-30,2004-02-01,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr A


#### Fechas

In [16]:
df2 = df2[df2['hasta'].str.contains('9999')]

In [17]:
split_date_column(df2,"hasta","dia_hasta","mes_hasta","year_hasta")

OutOfBoundsDatetime: ignored

### Juntar Informacion (Base para Modelo final)

### MODELOS MACHINE LEARNING

#### KNN

In [ ]:
def column_data_types(df):
    # Get the data types of each column
    data_types = df.dtypes

    # Create a DataFrame to store the results
    column_info = pd.DataFrame({
        'Column Name': data_types.index,
        'Data Type': data_types.values
    })

    return column_info


# Call the function to get column data types
column_info = column_data_types(df)

# Print the results
print(column_info)

            Column Name Data Type
0                    id     int64
1                 hijos     int64
2          edad_ingreso     int64
3           cp_vivienda    object
4            cp_trabajo    object
5            antiguedad     int64
6           edad_salida     int64
7            evaluacion   float64
8          distancia_km    object
9              tiempo_h    object
10          dia_ingreso     int64
11          mes_ingreso     int64
12         year_ingreso     int64
13           dia_salida     int64
14           mes_salida     int64
15          year_salida     int64
16       dia_nacimiento     int64
17       mes_nacimiento     int64
18      year_nacimiento     int64
19               Hombre     uint8
20                Mujer     uint8
21       baja_liverpool     uint8
22        baja_suburbia     uint8
23    Eventual No Sind.     uint8
24       Eventual Sind.     uint8
25      Planta No Sind.     uint8
26         Planta Sind.     uint8
27       SBB Eventuales     uint8
28           S

In [ ]:
# Assuming 'df' is your DataFrame
columns_to_drop = ['cp_vivienda', 'cp_trabajo']
df = df.drop(columns_to_drop, axis=1)


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score

# Define your feature columns and target variable
X = df[['hijos', 'edad_ingreso', 'evaluacion', 'dia_ingreso', 'mes_ingreso', 'year_ingreso', 'dia_salida', 'mes_salida', 'year_salida', 'dia_nacimiento', 'mes_nacimiento', 'year_nacimiento', 'Hombre', 'Mujer', 'baja_liverpool', 'baja_suburbia', 'Eventual No Sind.', 'Eventual Sind.', 'Planta No Sind.', 'Planta Sind.', 'SBB Eventuales', 'SBB Planta', 'Vía Planta No Sind.', 'Vía Planta Sind.', 'ubicacion_encoded', 'funcion_encoded']]
y = df['antiguedad']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a KNN regressor with the desired number of neighbors (e.g., 5)
knn_regressor = KNeighborsRegressor(n_neighbors=5)

# Train the KNN model on your training data
knn_regressor.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_regressor.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 1.218979983782169
Mean Absolute Error: 0.5848662028936025


In [39]:
from sklearn.metrics import r2_score

# Calculate R² (coefficient of determination)
r_squared = r2_score(y_test, y_pred)

print(f"R² (Coefficient of Determination): {r_squared:.2f}")

R² (Coefficient of Determination): 0.92
